# Agent Tools and Configurations

Tools give agents ability to take actions: sequential/parallel calls, error handling, and state persistence.

**What you'll learn:**
- Tools extend agent capabilities with actions
- @wrap_tool_call handles errors gracefully
- Agent selects tools based on docstrings
- Sequential vs parallel execution determined by model
- Clear tool descriptions guide selection

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [ ]:
# Custom Tool Name and Description
base_tools.web_search.name

base_tools.web_search.description

## Basic Agent with Tools

In [ ]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather]
)

response = agent.invoke({
    'messages': [HumanMessage("What's the weather in Tokyo?")]
})

response['messages'][-1].text

In [ ]:
# Agent without tools - single LLM node
agent = create_agent(model=model, tools=[])

response = agent.invoke({
    'messages': [HumanMessage("Hello")]
})

response['messages'][-1].text

## Sequential Tool Calls

In [ ]:
# Agent calls tools in sequence
response = agent.invoke({
    'messages': [HumanMessage(
        "Search for Apple news, then tell me weather in Cupertino"
    )]
})

response['messages'][-1].text

## Parallel Tool Calls

In [ ]:
# Agent may call tools in parallel (independent requests)
response = agent.invoke({
    'messages': [HumanMessage(
        "What's the weather in Paris and London?"
    )]
})

response['messages'][-1].text

## Tool Error Handling

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from langchain.tools import tool
# Tool WITHOUT internal error handling
@tool
def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b  # This will crash on division by zero

# Middleware catches it and returns graceful error to model
@wrap_tool_call
def handle_tool_errors(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}. Try different input.",
            tool_call_id=request.tool_call["id"]
        )


agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather, divide],
    middleware=[handle_tool_errors]
)

response = agent.invoke({
    'messages': [HumanMessage("what is 1/0?")]
})

response['messages'][-1].text

## Accessing Context - State Access

In [ ]:
from langchain.tools import ToolRuntime

# Tool that retrieves the count of messages in conversation
@tool
def get_message_count(runtime: ToolRuntime) -> str:
    """Get the total number of messages exchanged in the conversation."""
    messages = runtime.state["messages"]
    return f"Total messages in conversation: {len(messages)}"

# Tool that checks if a specific topic was discussed
@tool
def check_topic_discussed(topic: str, runtime: ToolRuntime) -> str:
    """Check if a specific topic was mentioned in the conversation."""
    messages = runtime.state["messages"]
    
    # Search through all messages for the topic
    for msg in messages:
        if hasattr(msg, 'content') and topic.lower() in str(msg.content).lower():
            return f"Yes, '{topic}' was discussed in the conversation."
    
    return f"No, '{topic}' has not been discussed yet."

agent = create_agent(
    model=model,
    tools=[get_message_count, check_topic_discussed]
)

response = agent.invoke({
    'messages': [
        HumanMessage("Let's talk about Python programming"),
        HumanMessage("Check if we discussed Python")
    ]
})

response['messages'][-1].text

## Accessing Context - Updating State with Command

In [ ]:
from langgraph.types import Command

# Update user_name in the agent state
@tool
def update_user_name(new_name: str, runtime: ToolRuntime) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

# Access custom state field
@tool
def get_user_preference(pref_name: str, runtime: ToolRuntime) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

agent = create_agent(
    model=model,
    tools=[update_user_name, get_user_preference]
)

response = agent.invoke({
    'messages': [HumanMessage("Update my name to Alice")]
})

response['messages'][-1].text

## Accessing Context - Runtime Context

In [ ]:
from dataclasses import dataclass

# Simulated user database
USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

agent = create_agent(
    model=model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

result['messages'][-1].text